<a href="https://colab.research.google.com/github/bbanzai88/Data-Science-Repository/blob/main/GGGP_Backtesting_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# GGGP — Risk-Aware Trading Rule Discovery (\$10,000 start + Portfolio)

This notebook discovers **interpretable, rule-based strategies** using **Grammar-Guided Genetic Programming (GGGP)**, optimized by **risk-aware backtesting**.  
It starts with **\$10,000** capital, reports **final amount** and **drawdown ($ and %) per ticker**, and builds a **best portfolio** from the top strategies.

**Pipeline**
1. Load OHLCV CSVs from `/content/sample_data` (Colab) or `/mnt/data/stocks` (local).
2. Compute indicators (SMA, EMA, RSI, MACD).
3. Evolve boolean trading rules via GGGP.
4. Backtest with costs and risk metrics (CAGR, MaxDD %, MaxDD $ , Calmar).
5. Rank tickers in a **leaderboard** (with currency columns).
6. Build an **equal-weight daily portfolio** from top-K strategies and evaluate.
7. Export leaderboard and portfolio equity to `/mnt/data/gggp_exports/`.


0) Get S&P Data

In [ ]:
# ✅ Robust S&P 500 downloader → /content/sample_data/*.csv
# - Tries Wikipedia with a real User-Agent
# - Falls back to two GitHub mirrors if needed
# - Normalizes Yahoo symbols (BRK.B -> BRK-B)
# - Downloads in chunks with yfinance

!pip -q install yfinance pandas lxml html5lib requests

import requests, pandas as pd, yfinance as yf, time, math
from pathlib import Path

OUTDIR = Path("/content/sample_data")
OUTDIR.mkdir(parents=True, exist_ok=True)

PERIOD = "10y"         # "5y", "10y", "max"
INTERVAL = "1d"        # "1d", "1wk", "1mo"
CHUNK_SIZE = 50        # batch size for yfinance multi-download
SLEEP_BETWEEN = 1.0    # seconds between chunks

def get_sp500_tickers():
    # 1) Wikipedia with headers
    wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/120.0 Safari/537.36"
        )
    }
    try:
        r = requests.get(wiki_url, headers=headers, timeout=20)
        r.raise_for_status()
        tables = pd.read_html(r.text)
        df = tables[0]
        # Find symbol column
        sym_col = None
        for c in df.columns:
            if str(c).lower().strip() in {"symbol", "ticker symbol"}:
                sym_col = c
                break
        if sym_col is None:
            raise RuntimeError("Symbol column not found on Wikipedia table.")
        df = df.rename(columns={sym_col: "Symbol"})
        tickers = df["Symbol"].astype(str).str.strip().tolist()
        if len(tickers) >= 450:  # sanity check
            print(f"✅ Wikipedia returned {len(tickers)} symbols.")
            return tickers
        else:
            print(f"⚠️ Wikipedia returned only {len(tickers)} rows, falling back…")
    except Exception as e:
        print(f"⚠️ Wikipedia fetch failed: {type(e).__name__}: {e}")

    # 2) GitHub mirror #1 (datahub)
    mirrors = [
        "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/main/data/constituents.csv",
        "https://raw.githubusercontent.com/datasets/s-and-p-500/master/data/constituents.csv",
    ]
    for url in mirrors:
        try:
            df = pd.read_csv(url)
            # Find col that looks like symbol
            sym_col = None
            for c in df.columns:
                if str(c).lower().strip() in {"symbol", "ticker", "ticker symbol"}:
                    sym_col = c
                    break
            if sym_col is None:
                continue
            tickers = df[sym_col].astype(str).str.strip().tolist()
            if len(tickers) >= 450:
                print(f"✅ Mirror worked: {url} ({len(tickers)} symbols)")
                return tickers
        except Exception as e:
            print(f"⚠️ Mirror failed {url}: {type(e).__name__}: {e}")

    raise RuntimeError("Could not retrieve S&P 500 tickers from any source.")

def normalize_for_yahoo(tickers):
    # BRK.B -> BRK-B, BF.B -> BF-B, etc.
    ysyms = [t.replace(".", "-").upper() for t in tickers if isinstance(t, str) and t.strip()]
    # de-duplicate while preserving order
    seen, out = set(), []
    for t in ysyms:
        if t not in seen:
            seen.add(t)
            out.append(t)
    return out

def save_ohlcv_csvs(ticker_list):
    saved, failed = [], []
    n = len(ticker_list)
    n_chunks = math.ceil(n / CHUNK_SIZE)
    for i in range(n_chunks):
        chunk = ticker_list[i*CHUNK_SIZE:(i+1)*CHUNK_SIZE]
        print(f"\nChunk {i+1}/{n_chunks}: downloading {len(chunk)} tickers…")
        try:
            data = yf.download(
                tickers=chunk,
                period=PERIOD,
                interval=INTERVAL,
                group_by="ticker",
                auto_adjust=False,
                threads=True,
                progress=False,
            )
        except Exception as e:
            print(f"⚠️ yfinance chunk error: {e}")
            failed.extend(chunk)
            time.sleep(SLEEP_BETWEEN)
            continue

        # Handle multiindex vs single-index frames
        for t in chunk:
            try:
                if isinstance(data.columns, pd.MultiIndex):
                    if t not in data.columns.get_level_values(0):
                        failed.append(t)
                        continue
                    df_t = data[t].copy()
                else:
                    # extremely rare in chunk mode
                    df_t = data.copy()

                df_t = df_t.reset_index()
                keep = [c for c in ["Date","Open","High","Low","Close","Volume"] if c in df_t.columns]
                if "Date" not in keep or "Close" not in keep:
                    failed.append(t); continue
                df_t = df_t[keep].dropna(subset=["Close"])
                outpath = OUTDIR / f"{t}.csv"
                df_t.to_csv(outpath, index=False)
                saved.append(t)
            except Exception as e:
                print(f"  ⚠️ Save failed {t}: {e}")
                failed.append(t)
        time.sleep(SLEEP_BETWEEN)
    return saved, failed

tickers_raw = get_sp500_tickers()
tickers = normalize_for_yahoo(tickers_raw)
print(f"\nUsing {len(tickers)} Yahoo-normalized tickers (first 15): {tickers[:15]}")

saved, failed = save_ohlcv_csvs(tickers)
print(f"\n✅ Saved {len(saved)} tickers to {OUTDIR}")
if failed:
    print(f"⚠️ Failed ({len(failed)}): {failed[:25]}{'...' if len(failed)>25 else ''}")



## 1) Setup & Data

In [ ]:

import os, glob, math, json, random, traceback
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Any, Tuple, List
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 100)

# Detect CSV directory
CAND_DIRS = [Path("/content/sample_data"), Path("/mnt/data/stocks")]
DATA_DIR = None
for d in CAND_DIRS:
    if d.exists() and any(Path(p).suffix.lower()==".csv" for p in glob.glob(str(d/"*.csv"))):
        DATA_DIR = d; break
if DATA_DIR is None:
    DATA_DIR = CAND_DIRS[0]
print("📁 Using data directory:", DATA_DIR)

START_CAPITAL = 10_000.0  # USD
PERIODS_PER_YEAR = 252

def load_ohlcv_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    rename = {}
    for need in ["Date","Open","High","Low","Close","Volume"]:
        found = None
        for c in df.columns:
            if c.lower()==need.lower():
                found=c; break
        if not found:
            raise ValueError(f"Missing column {need} in {path.name}")
        rename[found]=need
    df = df.rename(columns=rename)
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date").reset_index(drop=True)
    return df

# Load frames
frames: Dict[str, pd.DataFrame] = {}
for p in sorted(glob.glob(str(DATA_DIR/"*.csv"))):
    t = Path(p).stem.upper()
    try:
        df = load_ohlcv_csv(Path(p))
        if len(df) < 200:
            print(f"⚠️ {t}: too few rows ({len(df)}) — skipping")
            continue
        frames[t] = df
    except Exception as e:
        print(f"⚠️ Skipping {Path(p).name}: {e}")

tickers = sorted(frames.keys())
print(f"✅ Loaded {len(tickers)} tickers:", tickers[:20])


## 2) Indicators & Backtesting (currency-aware)

In [ ]:

# --- Indicators ---
def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False).mean()

def sma(series: pd.Series, window: int) -> pd.Series:
    return series.rolling(window=window, min_periods=window).mean()

def rsi(close: pd.Series, period: int = 14) -> pd.Series:
    delta = close.diff()
    up = np.where(delta > 0, delta, 0.0)
    down = np.where(delta < 0, -delta, 0.0)
    roll_up = pd.Series(up, index=close.index).rolling(period).mean()
    roll_down = pd.Series(down, index=close.index).rolling(period).mean()
    rs = roll_up / (roll_down + 1e-12)
    return 100 - (100 / (1 + rs))

def macd(close: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
    macd_line = ema(close, fast) - ema(close, slow)
    signal_line = ema(macd_line, signal)
    return macd_line, signal_line

def max_drawdown_amount(equity: pd.Series):
    """Returns (dd_pct negative, dd_amount negative)."""
    peak = equity.cummax()
    dd_amt = equity - peak
    dd_pct = dd_amt / (peak + 1e-12)
    return float(dd_pct.min()), float(dd_amt.min())

def cagr_from_equity(equity: pd.Series, periods_per_year: int = 252) -> float:
    if len(equity) < 2:
        return 0.0
    total = equity.iloc[-1] / equity.iloc[0] - 1.0
    years = max(len(equity)/periods_per_year, 1e-9)
    return (1 + total) ** (1/years) - 1

def simple_backtest(df: pd.DataFrame, signal: pd.Series, fee_bps: float = 5.0, slippage_bps: float = 0.0,
                    start_capital: float = 10_000.0, return_daily: bool = False):
    """Long/flat. Position=1 when signal True else 0. Costs on position changes."""
    close = df["Close"].astype(float)
    fee = fee_bps / 1e4
    slip = slippage_bps / 1e4

    pos = signal.astype(int).clip(0,1)
    pos_shift = pos.shift(1).fillna(0)
    trades = (pos != pos_shift).astype(int)

    daily_ret = close.pct_change().fillna(0.0)
    strat_ret = pos.shift(1).fillna(0) * daily_ret

    # costs on flip days
    cost = trades * (fee + slip)
    strat_ret -= cost

    equity = (1 + strat_ret).cumprod() * float(start_capital)

    dd_pct, dd_amt = max_drawdown_amount(equity)
    annual = cagr_from_equity(equity)

    calmar = annual / (abs(dd_pct) + 1e-9) if dd_pct < 0 else (float("inf") if annual > 0 else 0.0)

    return {
        "equity": equity,
        "final_amount": float(equity.iloc[-1]),
        "total_return_pct": float(equity.iloc[-1]/equity.iloc[0]-1.0),
        "cagr": float(annual),
        "max_dd_pct": float(dd_pct),
        "max_dd_amount": float(dd_amt),
        "calmar": float(calmar),
        "trades": int(trades.sum()),
        "daily_ret": strat_ret if return_daily else None,
    }


## 3) Grammar, Genome, and Rule Decoding

In [ ]:

from dataclasses import dataclass

INDICATOR_FAMILIES = [
    ("CLOSE", {}),
    ("SMA",   {"n": [5,10,20,50,100,200]}),
    ("EMA",   {"n": [5,10,20,50,100,200]}),
    ("RSI",   {"n": [7,14,21,28]}),
    ("MACD_LINE", {"fast":[8,12], "slow":[17,26], "signal":[9]}),
    ("MACD_SIGNAL", {"fast":[8,12], "slow":[17,26], "signal":[9]}),
]
OPS = [">","<"]
LOGICS = ["AND","OR"]
MAX_CLAUSES = 3

@dataclass
class Clause:
    left: tuple
    op: str
    right: tuple

@dataclass
class Rule:
    clauses: list
    joins: list

def random_indicator():
    fam, cfg = random.choice(INDICATOR_FAMILIES)
    params = {k: random.choice(v) for k, v in cfg.items()}
    return fam, params

def mutate_indicator(ind):
    fam, cfg = ind
    if random.random() < 0.3:
        return random_indicator()
    fam_def = next(x for x in INDICATOR_FAMILIES if x[0]==fam)
    _, pdef = fam_def
    new_params = dict(cfg)
    for k, choices in pdef.items():
        if random.random()<0.5:
            new_params[k] = random.choice(choices)
    return fam, new_params

def random_clause():
    return Clause(random_indicator(), random.choice(OPS), random_indicator())

def mutate_clause(cl: Clause) -> Clause:
    lft = mutate_indicator(cl.left) if random.random()<0.5 else cl.left
    rgt = mutate_indicator(cl.right) if random.random()<0.5 else cl.right
    op  = random.choice(OPS) if random.random()<0.2 else cl.op
    return Clause(lft, op, rgt)

def random_rule():
    k = random.randint(1, MAX_CLAUSES)
    cls = [random_clause() for _ in range(k)]
    jns = [random.choice(LOGICS) for _ in range(k-1)]
    return Rule(cls, jns)

def mutate_rule(rule: Rule, p_add=0.2, p_del=0.2) -> Rule:
    clauses = list(rule.clauses)
    joins = list(rule.joins)

    # mutate a random clause
    i = random.randrange(len(clauses))
    clauses[i] = mutate_clause(clauses[i])

    # maybe add a clause
    if len(clauses) < MAX_CLAUSES and random.random() < p_add:
        clauses.append(random_clause())
        if len(clauses) > 1:
            joins.append(random.choice(LOGICS))

    # maybe delete a clause
    if len(clauses) > 1 and random.random() < p_del:
        j = random.randrange(len(clauses))
        clauses.pop(j)
        if len(joins) >= 1:
            joins.pop(0 if j==0 else j-1)

    # tweak joins
    joins = [random.choice(LOGICS) if random.random()<0.2 else j for j in joins]

    return Rule(clauses=clauses, joins=joins)

def crossover_rule(a: Rule, b: Rule):
    if len(a.clauses)==1 and len(b.clauses)==1:
        ca, cb = a.clauses[0], b.clauses[0]
        if random.random()<0.5:
            ca2 = Clause(cb.left, ca.op, ca.right)
            cb2 = Clause(ca.left, cb.op, cb.right)
        else:
            ca2 = Clause(ca.left, ca.op, cb.right)
            cb2 = Clause(cb.left, cb.op, ca.right)
        return Rule([ca2],[]), Rule([cb2],[])
    cut_a = random.randrange(1, len(a.clauses)+1)
    cut_b = random.randrange(1, len(b.clauses)+1)
    na = a.clauses[:cut_a] + b.clauses[cut_b:]
    nb = b.clauses[:cut_b] + a.clauses[cut_a:]
    def rebuild(cls):
        return [] if len(cls)<=1 else [random.choice(LOGICS) for _ in range(len(cls)-1)]
    return Rule(na, rebuild(na)), Rule(nb, rebuild(nb))

def indicator_series(family: str, params: dict, df: pd.DataFrame) -> pd.Series:
    c = df["Close"].astype(float)
    if family=="CLOSE": return c
    if family=="SMA":   return sma(c, params["n"])
    if family=="EMA":   return ema(c, params["n"])
    if family=="RSI":   return rsi(c, params["n"])
    if family in ("MACD_LINE","MACD_SIGNAL"):
        m, s = macd(c, params["fast"], params["slow"], params["signal"])
        return m if family=="MACD_LINE" else s
    raise ValueError(f"Unknown family {family}")

def rule_to_signal(rule: Rule, df: pd.DataFrame) -> pd.Series:
    masks = []
    for cl in rule.clauses:
        L = indicator_series(cl.left[0], cl.left[1], df)
        R = indicator_series(cl.right[0], cl.right[1], df)
        m = (L > R) if cl.op == ">" else (L < R)
        masks.append(m.fillna(False))
    if not masks:
        return pd.Series(False, index=df.index)
    res = masks[0]
    for op, m in zip(rule.joins, masks[1:]):
        res = (res & m) if op=="AND" else (res | m)
    return res.fillna(False)

def rule_to_string(rule: Rule) -> str:
    parts = []
    for i, cl in enumerate(rule.clauses):
        def ind_str(ind): fam, p = ind; return f"{fam}{p}" if p else fam
        parts.append(f"({ind_str(cl.left)} {cl.op} {ind_str(cl.right)})")
        if i < len(rule.joins): parts.append(f" {rule.joins[i]} ")
    return "".join(parts)


## 4) GA Training / Evaluation (risk-aware fitness)

In [ ]:

@dataclass
class GAConfig:
    pop_size: int = 60
    gens: int = 25
    tourn_k: int = 3
    cx_prob: float = 0.7
    mut_prob: float = 0.5
    seed: int = 42
    fee_bps: float = 5.0
    slippage_bps: float = 0.0
    fitness_lambda_dd: float = 2.0
    train_frac: float = 0.7

def fitness(rule: Rule, df: pd.DataFrame, cfg: GAConfig):
    sig = rule_to_signal(rule, df)
    res = simple_backtest(df, sig, cfg.fee_bps, cfg.slippage_bps, start_capital=START_CAPITAL)
    # Risk-aware fitness: CAGR - lambda * |max_dd_pct|
    score = res["cagr"] - cfg.fitness_lambda_dd * abs(res["max_dd_pct"])
    return score, res

def train_ga_on_stock(df: pd.DataFrame, cfg: GAConfig):
    n = len(df)
    split = int(n * cfg.train_frac)
    train = df.iloc[:split].copy()
    test  = df.iloc[split:].copy()

    random.seed(cfg.seed)
    pop = [random_rule() for _ in range(cfg.pop_size)]

    def tournament():
        cand = random.sample(pop, cfg.tourn_k)
        scored = [(fitness(r, train, cfg)[0], r) for r in cand]
        return max(scored, key=lambda x:x[0])[1]

    best_rule, best_score = None, -1e9
    history = []
    for g in range(cfg.gens):
        # evaluate
        for r in pop:
            s, _ = fitness(r, train, cfg)
            if s > best_score:
                best_rule, best_score = r, s
        history.append(best_score)

        # next generation
        new_pop = []
        while len(new_pop) < cfg.pop_size:
            if random.random() < cfg.cx_prob:
                p1, p2 = tournament(), tournament()
                c1, c2 = crossover_rule(p1, p2)
                if random.random() < cfg.mut_prob: c1 = mutate_rule(c1)
                if random.random() < cfg.mut_prob: c2 = mutate_rule(c2)
                new_pop.extend([c1, c2])
            else:
                p = tournament()
                c = mutate_rule(p) if random.random()<cfg.mut_prob else p
                new_pop.append(c)
        pop = new_pop[:cfg.pop_size]

    # Evaluate best on both sets (with daily returns for portfolio)
    train_sig = rule_to_signal(best_rule, train)
    test_sig  = rule_to_signal(best_rule,  test)
    train_res = simple_backtest(train, train_sig, cfg.fee_bps, cfg.slippage_bps, START_CAPITAL, return_daily=True)
    test_res  = simple_backtest(test,  test_sig,  cfg.fee_bps, cfg.slippage_bps, START_CAPITAL, return_daily=True)

    return {
        "best_rule": best_rule,
        "best_rule_str": rule_to_string(best_rule),
        "train_metrics": train_res,
        "test_metrics": test_res,
        "split_idx": split,
        "history": history,
    }


## 5) Run Evolution Across Tickers + Leaderboard (with $ columns)

In [ ]:

cfg = GAConfig()

results = []
per_ticker = {}

for t in tickers:
    print(f"=== {t} ===")
    try:
        out = train_ga_on_stock(frames[t], cfg)
        per_ticker[t] = out
        tm = out["test_metrics"]
        results.append({
            "ticker": t,
            "rule": out["best_rule_str"],
            "test_final_amount": tm["final_amount"],
            "test_total_return_pct": tm["total_return_pct"],
            "test_cagr": tm["cagr"],
            "test_max_dd_pct": tm["max_dd_pct"],
            "test_max_dd_amount": tm["max_dd_amount"],
            "test_calmar": tm["calmar"],
            "test_trades": tm["trades"],
        })
    except Exception as e:
        print(f"  ⚠️ {t} failed: {e}")

leaderboard = pd.DataFrame(results).sort_values(
    ["test_calmar","test_cagr","test_final_amount"], ascending=[False, False, False]
).reset_index(drop=True)

leaderboard.head(20)


## 6) Plot Top Strategy Equity (currency)

In [ ]:

if len(leaderboard) > 0:
    t = leaderboard.iloc[0]["ticker"]
    out = per_ticker[t]
    split = out["split_idx"]
    df = frames[t]
    rule = out["best_rule"]
    sig = rule_to_signal(rule, df)
    res = simple_backtest(df, sig, cfg.fee_bps, cfg.slippage_bps, START_CAPITAL)

    print(f"🏆 Top: {t}")
    print("Rule:", out["best_rule_str"])
    print(f"Final = ${res['final_amount']:,.2f} | MaxDD = {res['max_dd_pct']:.2%} ({res['max_dd_amount']:,.0f}) | CAGR = {res['cagr']:.2%} | Calmar = {res['calmar']:.2f}")

    eq = res["equity"]
    plt.figure(figsize=(10,4))
    plt.plot(df["Date"], eq.values)
    plt.axvline(df["Date"].iloc[split], linestyle="--")
    plt.title(f"Equity — {t}")
    plt.xlabel("Date"); plt.ylabel("Equity ($)")
    plt.show()
else:
    print("No leaderboard results.")


## 7) Build **Best Portfolio** (equal-weight daily over Top-K)

In [ ]:

TOP_K = min(10, len(leaderboard))  # change as desired
selected = leaderboard.head(TOP_K)["ticker"].tolist()
print(f"🧺 Portfolio members ({len(selected)}):", selected)

def build_member_daily_returns(ticker: str, out_obj: dict, df_full: pd.DataFrame):
    """Recompute full-period daily returns for the evolved rule on full data (train+test)."""
    rule = out_obj["best_rule"]
    sig = rule_to_signal(rule, df_full)
    bt = simple_backtest(df_full, sig, cfg.fee_bps, cfg.slippage_bps, START_CAPITAL, return_daily=True)
    ret = bt["daily_ret"].rename(ticker)
    ret.index = df_full["Date"]
    return ret

if selected:
    rets = []
    for t in selected:
        r = build_member_daily_returns(t, per_ticker[t], frames[t])
        rets.append(r)

    R = pd.concat(rets, axis=1).sort_index()
    port_ret = R.mean(axis=1, skipna=True).fillna(0.0)
    port_equity = (1 + port_ret).cumprod() * START_CAPITAL

    # Portfolio metrics
    def max_dd_amt(e):
        peak = e.cummax(); dd_amt = e - peak; dd_pct = dd_amt/peak
        return float(dd_pct.min()), float(dd_amt.min())
    ddp, dda = max_dd_amt(port_equity)
    years = max(len(port_equity)/252, 1e-9)
    total = port_equity.iloc[-1]/port_equity.iloc[0]-1
    cagr_port = (1+total)**(1/years)-1
    calmar_port = cagr_port / (abs(ddp)+1e-9) if ddp<0 else (float("inf") if cagr_port>0 else 0.0)

    print(f"📈 Portfolio (Top {len(selected)}): Final=${port_equity.iloc[-1]:,.2f} | MaxDD={ddp:.2%} ({dda:,.0f}) | CAGR={cagr_port:.2%} | Calmar={calmar_port:.2f}")

    plt.figure(figsize=(10,4))
    plt.plot(port_equity.index, port_equity.values)
    plt.title(f"Best Portfolio — Top {len(selected)} by Calmar")
    plt.xlabel("Date"); plt.ylabel("Equity ($)")
    plt.show()
else:
    print("No members selected for portfolio.")


## 8) Export artifacts

In [ ]:

OUT_DIR = Path("/mnt/data/gggp_exports"); OUT_DIR.mkdir(parents=True, exist_ok=True)

if 'leaderboard' in globals() and len(leaderboard)>0:
    lb_path = OUT_DIR / "leaderboard_with_currency.csv"
    leaderboard.to_csv(lb_path, index=False)
    print("💾 Saved leaderboard:", lb_path)

    try:
        port_df = pd.DataFrame({"Date": port_equity.index, "Equity": port_equity.values})
        port_path = OUT_DIR / "best_portfolio_equity.csv"
        port_df.to_csv(port_path, index=False)
        print("💾 Saved best portfolio equity:", port_path)
    except NameError:
        print("ℹ️ Portfolio not built (skipping save).")
else:
    print("ℹ️ No leaderboard to export.")
